### Run the following chunk of code to import any libraries or packages required to run the rest of the code.

In [ ]:
import numpy as np
import os as os
import matplotlib.pyplot as plt
import io
import PIL
import gc

### Run the following chunk of code to create trajectory gifs.
This code will take as an input .npy files which contain the positions of each amino acid in a simulation for each time step and convert them into a simulation gif. The gif will project the position of each amino acid onto one of the faces of the simulation slab, and color each of the proteins a different color. Note, the user will need to update the directory variables to account for their own directory setup.

In [ ]:
base_Directory = ""

phos = 4
input_Working_Directory = base_Directory + "Inputs/" + str(phos) + "x_Phos/"
output_Working_Directory = base_Directory + "Outputs/" + str(phos) + "x_Phos/"

chain_Length = 150
box_Top = 30
box_Bottom = -30
box_Right = 600
box_Left = -600

for file in os.listdir(input_Working_Directory):
    if file.replace("Positions.npy","GIF.gif") in os.listdir(output_Working_Directory):
        continue
    else:
        print(file)
        positions = np.load(input_Working_Directory + file)
        protein_Chains = int(np.shape(positions)[1]/chain_Length)
        red = [[1,0,0]]
        orange = [[1,0.647,0]]
        yellow = [[0,0,0]]
        green = [[0,1,0]]
        blue = [[0,0,1]]
        purple = [[0.627,0.125,0.941]]
        color_Count = 0
        colors = []
        for i in range(protein_Chains):
          if color_Count == 0:
            colors += red*chain_Length
          elif color_Count == 1:
            colors += orange*chain_Length
          elif color_Count == 2:
            colors += yellow*chain_Length
          elif color_Count == 3:
            colors += green*chain_Length
          elif color_Count == 4:
            colors += blue*chain_Length
          elif color_Count == 5:
            colors += purple*chain_Length
          color_Count += 1
          if color_Count == 6:
            color_Count = 0
        frames = []
        for i in range(len(positions)):
          if i%100 == 0:
            print(i)
          fig = plt.figure(figsize=(20, 1), dpi=100)
          ax = plt.axes()
          plt.scatter(positions[i][:,2],positions[i][:,1],color = colors,s = 0.6, linewidth = 0.1,edgecolor = "black")
          plt.axvline(x = box_Left, color = 'black', lw = 2)
          plt.axvline(x = box_Right, color = 'black', lw = 2)
          plt.axhline(y = box_Bottom, color = "black", lw = 2)
          plt.axhline(y = box_Top, color = "black", lw = 2)
          plt.xlim(box_Left,box_Right)
          plt.ylim(box_Bottom,box_Top)
          plt.text(box_Left + 10,box_Top - 10,"Time = " + str(np.round((1/1000)*i,4)) + r"$ \mu s$")
          plt.axis("off")
          buf = io.BytesIO()
          fig.savefig(buf)
          buf.seek(0)
          frames.append(PIL.Image.open(buf))
          plt.clf()
          plt.close("all")
          gc.collect()
        frames[0].save(output_Working_Directory + file.replace("Positions.npy","GIF.gif"), save_all=True, append_images=frames[1:], optimize=False, duration=100, loop=0)
        del frames
        del positions
        gc.collect()